In [2]:
# import classification.convenience_functions.mkl_base as mkl_base
from aknotebooks.classification.convenience_functions.mkl_base import hardDrivesLoc, dataOnlyDrive, folderList, symbols, \
    dataList, symbolFeaturesLocation, selection,checkDir
from aknotebooks.classification.convenience_functions.fit_mkl import featureCreation
from sklearn.metrics.pairwise import rbf_kernel as RBF

ModuleNotFoundError: No module named 'classification'

In [ ]:
import MKLpy
from MKLpy.metrics import pairwise
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
from MKLpy.metrics import pairwise
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.preprocessing import kernel_normalization
from MKLpy.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics.pairwise import rbf_kernel as RBF
import os
import pandas as pd
import numpy as np
import pickle as pkl

In [ ]:
symbols

In [ ]:

symbolIdx = 2  # pick one of the symbols

# do a join to get the location
# symbolLocation = "/".join((finalLocation, symbols[symbolIdx]))
# # get the features now

selectionLoc = os.path.join(hardDrivesLoc, selection)

# ''' location of WorkDrive'''
# dataList = [s for s in os.listdir(selectionLoc) if s.startswith('Dat')]
# path = 'MKL_Experiments'
MKLExpPath = os.path.join(os.path.join(hardDrivesLoc, selection, dataList[1]), 'MKL_Experiments')
MKLSymbolPath = os.path.join(MKLExpPath, symbols[symbolIdx])
MKLSymbolKernelsPath = "/".join((MKLSymbolPath, 'Kernels'))

SymbolCommonPaths = mkl_base.open_pickle_file(MKLSymbolPath, 'LocDictsListCorrect.pkl')
uniqueTrainingKeys = np.unique([i[1] for i in SymbolCommonPaths])

In [ ]:

kernelDates = [file.split("_")[0] for file in os.listdir('/media/ak/WorkDrive/Data/MKL_Experiments/PRU.L/Kernels/')]

                

In [ ]:
kernelDates[0]

In [ ]:
keys=list(SymbolCommonPaths.keys())
type(keys)
# for idx, key in enumerate(keys):
#     print(idx)
#     print(key)
#     kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((key[0], "_Kernels.pkl"))))
#     checkDir(kernelFileName)
#     print(kernelFileName)
#     print(SymbolCommonPaths[key])
i=kernelDates[0]
result=[]
for j in keys:
    if i in j:
        result.append(j)

print(result)

In [ ]:
print(SymbolCommonPaths[keys[920]])
kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((i, "_Kernels.pkl"))))
checkDir(kernelFileName)

In [ ]:
featuresTupleFile=pkl.load(open(SymbolCommonPaths[result[1]][0], "rb"),encoding='latin1')


In [ ]:
labelsDf  = pd.read_csv(SymbolCommonPaths[result[1]][1])


In [ ]:
''' pop the labels out'''
labels = labelsDf['label_PrMov__window_5__thres_arbitrary__0.1']
''' and same with features'''
dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1], \
                            featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)

In [ ]:


'''dataframe of Features and Labels - X and Y'''
dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
''' drop the labels from the features'''
dfX = dfXY.drop(columns=[labelName])
arrX = np.array(dfX)
''' feature normalisation'''
# feature scaling in [0,1] - X = rescale_01(arrX)
X = normalization(rescale_01(arrX))
y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
''' returns features, labels'''

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
KLtr = [pairwise.homogeneous_polynomial_kernel(X, degree=d) for d in range(4)]
''' Compute RBF Kernels'''
gamma_range = np.logspace(-9, 3, 13)
ker_list = [RBF(X, gamma=g) for g in gamma_range]

## should get the components to have a shape X.shape x y.shape

In [ ]:
## try some bullshit test ###
clf = AverageMKL().fit(KLtr,y) #a wrapper for averaging kernels
print(clf.weights) #print the weights of the combination of base kernels
print ('training EasyMKL...for polynomials and RBF', end='')
clfEasy = EasyMKL(lam=0.1).fit(KLtr,y)#combining kernels with the EasyMKL algorithm

In [ ]:
## and test the bullshit of ker_list ##
clfRBF = EasyMKL(lam=0.1).fit(ker_list,y) 

In [ ]:
KernelsTest = [KLtr, ker_list]
testFileName = 'KernelTestPickle.pkl'
PapersPath = os.getcwd()
pkl.dump(KernelsTest, open("/".join((PapersPath, testFileName)), "wb"))

In [ ]:
loadPickleKernels = pkl.load(open("/".join((PapersPath, testFileName)), "rb"))

In [ ]:
KLtrloaded = loadPickleKernels[1]
ker_list_loaded= loadPickleKernels[1]

In [ ]:
## try some bullshit test ###
clfloaded = AverageMKL().fit(KLtrloaded,y) #a wrapper for averaging kernels
clfEasyloaded = EasyMKL(lam=0.1).fit(KLtrloaded,y)#combining kernels with the EasyMKL algorithm
print('test complete')

In [ ]:
 ### and train 3 classifiers ###
    clf = AverageMKL().fit(KLtr,ytr) #a wrapper for averaging kernels
    print(clf.weights) #print the weights of the combination of base kernels
    print ('training EasyMKL...for polynomials and RBF', end='')
    clfEasy = EasyMKL(lam=0.1).fit(KLtr,ytr)#combining kernels with the EasyMKL algorithm
    clfRBF = EasyMKL(lam=0.1).fit(ker_list,ytr)

In [ ]:
if __name__ == '__main__':
    trainingKey =1 
    keyDate = list(SymbolCommonPaths)[trainingKey]
    
    

    for trainingKey, keyDate in enumerate(list(SymbolCommonPaths)):

        print(trainingKey, keyDate[1])
        Xtr, ytr = featureCreation(idxKey=trainingKey,
                                   locDict=SymbolCommonPaths)  # need to refactor this to read from the mkl_base
        if Xtr.shape[0] == ytr.shape[0]:
            print(list(SymbolCommonPaths.keys())[trainingKey][0])
            kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((keyDate[0], "_Kernels.pkl"))))
            loadPickleKernels = pkl.load(open(kernelFileName, "rb"))
            print('doing the kernels bit')
            KLsimple = loadPickleKernels[0]
            KLrbf = loadPickleKernels[1]
            clf = AverageMKL().fit(KLsimple, ytr)  # a wrapper for averaging kernels
            clfEasy = EasyMKL(lam=0.1).fit(KLsimple, ytr)#combining kernels with the EasyMKL algorithm
            clfRBF = EasyMKL(lam=0.1).fit(KLrbf, ytr)
            print('------')
            print('finished MKL Fitting- Now will test Predictions')
            print('Average Kernel Testing')
            y_pred = clf.predict(KLsimple)  # predictions
            y_score = clf.decision_function(KLsimple)  # rank
            accuracy = accuracy_score(ytr, y_pred)
            fprAverage, tprAverage, thresholdsAverage = roc_curve(ytr.ravel(), y_score.ravel())
            roc_auc = auc(fprAverage, tprAverage)
            print('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))
        else:
            print('Shapes dont match.')
            continue

In [ ]:
trainingKey =1 
keyDate = list(SymbolCommonPaths)[trainingKey]
print(trainingKey, keyDate[1])
Xtr, ytr = featureCreation(idxKey=trainingKey,
                                   locDict=SymbolCommonPaths)  # need to refactor

In [ ]:
Xtr.shape[0] == ytr.shape[0]

In [ ]:
print(list(SymbolCommonPaths.keys())[trainingKey][0])
kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((keyDate[1], "_Kernels.pkl"))))
loadPickleKernels = pkl.load(open(kernelFileName, "rb"))
print('doing the kernels bit')
KLsimple = loadPickleKernels[0]
KLrbf = loadPickleKernels[1]

In [ ]:
clf = AverageMKL().fit(KLsimple, ytr)  # a wrapper for averaging kernels
clfEasy = EasyMKL(lam=0.1).fit(KLsimple, ytr)#combining kernels with the EasyMKL algorithm
            
clfRBF = EasyMKL(lam=0.1).fit(KLrbf, ytr)